In [15]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib
from pprint import pprint as print
import hvplot.pandas

pd.options.plotting.backend = 'holoviews'
matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_format = 'svg' 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

cred = {"host": 'localhost', 'dbname': 'yukontaf', 'user': 'glebsokolov', 'password': ''}
from sqlalchemy import create_engine

con = create_engine(
    f'postgresql://{cred["user"]}:{cred["password"]}@{cred["host"]}/{cred["dbname"]}'
)
def select(sql):
    return pd.read_sql(sql, con)

In [24]:
import io, pickle, tqdm, mmap

fname = "/Users/glebsokolov/Datasets/wikiNewsVec.vec"
with open(fname, mode="r", encoding="utf-8") as file_obj:
    with mmap.mmap(file_obj.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_obj:
        data = {}
        for _ in tqdm.trange(999000):
            line = mmap_obj.readline()
            # tokens = line.rstrip().split(' ')
            word = str(line).split(" ")[0].replace("b'", "")
            vec = str(line).split(" ")[1:]
            vec[-1] = vec[-1].replace('\\n"', "").replace("\\n'", "")
            data[word] = list(map(float, vec))


100%|██████████| 999000/999000 [03:39<00:00, 4556.46it/s]


In [11]:
import io, pickle, tqdm, mmap

fname = "/Volumes/ENVIRONMENT/Datasets/wikiNewsVec.vec"


def load_vectors(fname):
    fin = io.open(fname, "r", encoding="utf-8", newline="\n", errors="ignore")
    num_lines = sum(1 for _ in io.open(fname))
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm.tqdm(fin):
        tokens = line.rstrip().split(" ")
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data


vectors = load_vectors(fname)


999994it [02:04, 8058.88it/s] 


In [2]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000
)
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])


In [3]:
import numpy as np


def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results


x_train = vectorize_sequences(train_data, 10000)
x_test = vectorize_sequences(test_data, 10000)


In [4]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)


In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(46, activation="softmax"))


2022-05-24 20:09:34.688321: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)


In [7]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


In [9]:
model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val),
)


Epoch 1/20
16/16 [==============================] - 2s 47ms/step - loss: 2.5897 - accuracy: 0.5279 - val_loss: 1.7161 - val_accuracy: 0.6450
Epoch 2/20
16/16 [==============================] - 0s 31ms/step - loss: 1.4009 - accuracy: 0.7081 - val_loss: 1.2863 - val_accuracy: 0.7140
Epoch 3/20
16/16 [==============================] - 1s 32ms/step - loss: 1.0415 - accuracy: 0.7789 - val_loss: 1.1110 - val_accuracy: 0.7640
Epoch 4/20
16/16 [==============================] - 1s 37ms/step - loss: 0.8209 - accuracy: 0.8241 - val_loss: 1.0295 - val_accuracy: 0.7710
Epoch 5/20
16/16 [==============================] - 0s 30ms/step - loss: 0.6532 - accuracy: 0.8626 - val_loss: 0.9739 - val_accuracy: 0.7960
Epoch 6/20
16/16 [==============================] - 1s 31ms/step - loss: 0.5247 - accuracy: 0.8899 - val_loss: 0.9305 - val_accuracy: 0.8070
Epoch 7/20
16/16 [==============================] - 0s 31ms/step - loss: 0.4207 - accuracy: 0.9136 - val_loss: 0.9024 - val_accuracy: 0.8150
Epoch 8/20
16

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = 300
training_samples = 50000


In [34]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results


In [18]:
from fast_ml.model_development import train_valid_test_split

max_words = 10000

df = pd.read_feather("/Volumes/ENVIRONMENT/Datasets/amazonSentiment.feather")
sample = df.sample(60000)
sample.rating = sample.rating.apply(lambda x: int(float(x)))
tokenizer = Tokenizer(num_words=max_words)
texts, labels = sample.text, sample.rating
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(sample.text)
data = pad_sequences(sequences, maxlen=maxlen)


In [19]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = to_categorical(labels)[indices]


In [47]:
validation_samples = 10000
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples : training_samples + validation_samples]
y_val = labels[training_samples : training_samples + validation_samples]


In [53]:
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_val)


In [50]:
y_train


array([[0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [43]:
y_train, y_test = to_categorical(y_train), to_categorical(y_val)


In [21]:
embedding_dim = 300
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [31]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(6, activation="softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 300)          3000000   
                                                                 
 flatten_3 (Flatten)         (None, 90000)             0         
                                                                 
 dense_10 (Dense)            (None, 256)               23040256  
                                                                 
 dense_11 (Dense)            (None, 6)                 1542      
                                                                 
Total params: 26,041,798
Trainable params: 23,041,798
Non-trainable params: 3,000,000
_________________________________________________________________


In [60]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(6, activation="softmax"))


In [54]:
x_train.shape


(50000, 10000)

In [61]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])


In [66]:
# twitter hate speech
train_path, test_path = (
    "/Volumes/ENVIRONMENT/Datasets/twitterHateSpeech/train.csv",
    "/Volumes/ENVIRONMENT/Datasets/twitterHateSpeech/test.csv",
)
train, test = pd.read_csv(train_path), pd.read_csv(test_path)
df = pd.concat([train, test])

df.reset_index().dropna().to_feather(
    "/Volumes/ENVIRONMENT/Datasets/twitterHateSpeech/combined.feather"
)


In [68]:
df.info()
df.label.value_counts()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 49159 entries, 0 to 17196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


0.000    29720
1.000     2242
Name: label, dtype: int64

In [63]:
model.fit(x_train, y_train, epochs=30, batch_size=16, validation_data=(x_test, y_val))


Epoch 1/30
3125/3125 [==============================] - 66s 21ms/step - loss: 0.2578 - acc: 0.9153 - val_loss: 3.1684 - val_acc: 0.4178
Epoch 2/30
1946/3125 [=================>............] - ETA: 23s - loss: 0.2051 - acc: 0.9371

KeyboardInterrupt: 

In [2]:
data = tc.SFrame.read_csv(
    "/Users/glebsokolov/Dropbox/Datasets/amazon_review_full_csv/train.csv", header=False
)
data = data.rename({"X1": "label", "X2": "sup", "X3": "text"})


Unexpected characters after last column. ""The Nightmare Before Christmas...""
Parse failed at token ending at: 
	ll work for me! :\"
"3","The Nightmare Before Christmas...",^"Burton doesn't falter in his imaginative screen
Successfully parsed 3 tokens: 
	0: 1
	1: I'm such a dummy
	2: I didn't p ... or me! :"

Unexpected characters after last column. ""Excellent product""
Parse failed at token ending at: 
	bly will not purchase it again. :\"
"5","Excellent product",^"I needed an Omega 3 product that would have the
Successfully parsed 3 tokens: 
	0: 2
	1: I like it, ... n don't :|
	2: I love thi ... again. :"

Unexpected characters after last column. ""How many different plot lines at the same time?""
Parse failed at token ending at: 
	 =o\"
"2","How many different plot lines at the same time?",^"Wow, truly how many plot lines can you put into
Successfully parsed 3 tokens: 
	0: 1
	1: Terrible m ... t anyway."
	2: What to sa ... ther. =o"

3 lines failed to parse correctly

Finished parsing file /Users/glebsokolov/Dropbox/Datasets/amazon_review_full_csv/train.csv

...

Read 993072 lines. Lines per second: 79565.5

Read 1459185 lines. Lines per second: 82432.7

73 lines failed to parse correctly

Finished parsing file /Users/glebsokolov/Dropbox/Datasets/amazon_review_full_csv/train.csv

Parsing completed. Parsed 1611764 lines in 20.1397 secs.

In [8]:
training_data, test_data = data.random_split(0.8)

model = tc.text_classifier.create(
    training_data, "label", features=["text"], max_iterations=100
)


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 1224818

Number of classes           : 5

...

| 0         | 2        | 1.000000  | 6.484780     | 0.450113          | 0.361964            |

| 1         | 4        | 1.000000  | 639.285837   | 0.497165          | 0.398588            |

| 2         | 6        | 1.000000  | 1320.990155  | 0.542041          | 0.416164            |

| 3         | 7        | 1.000000  | 1973.481452  | 0.556462          | 0.422229            |

| 4         | 8        | 1.000000  | 2635.285269  | 0.569433          | 0.425549            |

In [7]:
data


label,sup,text
3,more like funchuck,Gave this to my dad for agag gift after directing ...
5,Inspiring,I hope a lot of peoplehear this cd. We need ...
5,The best soundtrack everto anything. ...,I'm reading a lot ofreviews saying that this ...
4,Chrono Cross OST,The music of YasunoriMisuda is without ...
5,Too good to be true,Probably the greatestsoundtrack in history! ...
5,There's a reason for theprice ...,"There's a reason this CDis so expensive, even ..."
1,Buyer beware,"This is a self-publishedbook, and if you want to ..."
4,"Errors, but great story",I was a dissapointed tosee errors on the back ...
1,The Worst!,"A complete waste of time.Typographical errors, ..."
1,Oh please,I guess you have to be aromance novel lover for ...


In [32]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])


In [33]:
model.fit(x_train, y_train, epochs=30, batch_size=1024, validation_data=(x_val, y_val))


Epoch 1/30
49/49 [==============================] - 48s 983ms/step - loss: 1.6636 - acc: 0.2569 - val_loss: 1.5965 - val_acc: 0.2748
Epoch 2/30
49/49 [==============================] - 51s 1s/step - loss: 1.4918 - acc: 0.3513 - val_loss: 1.5607 - val_acc: 0.2989
Epoch 3/30
49/49 [==============================] - 49s 997ms/step - loss: 1.3979 - acc: 0.4124 - val_loss: 1.5238 - val_acc: 0.3329
Epoch 4/30
49/49 [==============================] - 51s 1s/step - loss: 1.3061 - acc: 0.4675 - val_loss: 1.4870 - val_acc: 0.3469
Epoch 5/30
49/49 [==============================] - 48s 989ms/step - loss: 1.2165 - acc: 0.5215 - val_loss: 1.5523 - val_acc: 0.3361
Epoch 6/30
49/49 [==============================] - 50s 1s/step - loss: 1.1277 - acc: 0.5603 - val_loss: 1.5476 - val_acc: 0.3438
Epoch 7/30
49/49 [==============================] - 43s 878ms/step - loss: 1.0519 - acc: 0.5972 - val_loss: 1.5548 - val_acc: 0.3576
Epoch 8/30
49/49 [==============================] - 45s 911ms/step - loss: 0.9

KeyboardInterrupt: 

In [51]:
path = "/Volumes/Environment/Datasets/preTrainedWikiNews.h5"
model.save_weights(path)


In [14]:
model.load_weights("/Volumes/ENVIRONMENT/Datasets/preTrainedWikiNews.h5")
